In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType

In [ ]:
%run "/Workspace/Users/cronies-02gorilla@icloud.com/access"

key,value
spark.databricks.delta.formatCheck.enabled,false


In [ ]:
stream_df = spark \
.readStream \
.format('kinesis') \
.option('streamName','streaming-0affea73130b-geo') \
.option('initialPosition','latest') \
.option('region','us-east-1') \
.option('awsAccessKey', ACCESS_KEY) \
.option('awsSecretKey', SECRET_KEY) \
.load()

In [ ]:
# Define the schema to speed up processing
jsonSchema = StructType([ 
                         StructField("ind", StringType(), True), 
                         StructField("country", StringType(), True), 
                         StructField("latitude", StringType(), True), 
                         StructField("longitude", StringType(), True), 
                         StructField("timestamp", StringType(), True), 
                         ])

In [ ]:
string_df = stream_df.selectExpr("CAST(data as STRING)")

In [ ]:
json_df = string_df.select(from_json(col("data"), jsonSchema))

In [ ]:
geo_stream_df = json_df.withColumn("ind", col("from_json(data)")["ind"])\
.withColumn("country", col("from_json(data)")["country"])\
.withColumn("latitude", col("from_json(data)")["latitude"])\
.withColumn("longitude", col("from_json(data)")["longitude"])\
.withColumn("timestamp", col("from_json(data)")["timestamp"])


In [ ]:
# drop from_json(data) column
cleaned_df = geo_stream_df.drop("from_json(data)")

# drop duplicates
cleaned_df = cleaned_df.dropDuplicates()

# Create a new column coordinates that contains an array based on the latitude and longitude columns
cleaned_df = cleaned_df.withColumn("coordinates", array("latitude", "longitude"))

# Drop the latitude and longitude columns from the DataFrame
cleaned_df = cleaned_df.drop("latitude", "longitude")

# cast int datatype to ind
cleaned_df = cleaned_df.withColumn("ind", cleaned_df["ind"].cast("int"))

# Convert the timestamp column from a string to a timestamp data type
cleaned_df = cleaned_df.withColumn("timestamp", to_timestamp("timestamp"))

# drop duplicates
cleaned_df = cleaned_df.dropDuplicates()

# reorder the DataFrame columns
cleaned_df = cleaned_df.select("ind", "country", "coordinates", "timestamp")

In [ ]:
output_geo_df = cleaned_df

In [ ]:
display(output_geo_df)

ind,country,coordinates,timestamp
709,Albania,"List(-88.8298, -170.188)",2020-12-03T15:25:22.000+0000
7078,Fiji,"List(1.43641, 135.772)",2019-08-25T06:50:19.000+0000
8083,Saint Martin,"List(68.4336, -67.038)",2018-07-03T01:16:02.000+0000
10868,Panama,"List(-42.1051, 153.465)",2019-06-15T06:59:12.000+0000
7596,Albania,"List(-87.2, -177.109)",2022-09-03T15:27:36.000+0000
9402,Argentina,"List(-89.4739, -176.154)",2022-06-15T08:25:16.000+0000
462,Argentina,"List(-89.63, -179.022)",2022-08-19T23:59:15.000+0000
3647,American Samoa,"List(-89.3099, -166.033)",2018-11-27T04:18:51.000+0000
3480,American Samoa,"List(-89.3099, -166.033)",2018-04-10T08:49:49.000+0000
10920,Egypt,"List(-58.5064, -149.458)",2021-05-05T08:08:57.000+0000


In [ ]:
output_geo_df.writeStream \
  .format("delta") \
  .queryName("0affea73130b_geo_query") \
  .outputMode("append") \
  .option("checkpointLocation", "/tmp/kinesis/_checkpoints/0affea73130b_geo_query") \
  .table("0affea73130b_geo_table")

Out[34]: <pyspark.sql.streaming.StreamingQuery at 0x7fa002f3f760>

In [ ]:
# dbutils.fs.rm("/tmp/kinesis/_checkpoints/0affea73130b_geo_query", True)